In [2]:
from IPython.core.display import *
from StringIO import StringIO
from __future__ import division # 1 / 2 = 0.5 instead of 0
from operator import *
import copy
import itertools

## S-Expressions

With that out of the way, we need a language with which to express our constants, variables and predicates and that language will be based on s-expressions.

**constants** - There are two types of constants, values and predicates. Values should start with an uppercase letter. Fred is a constant value, so is Barney and Food. Predicates are named using lowercase letters. loves is a predicate and so is hates. This is only a convention. Secret: your code does not need to treat these two types of constants differently.

**variables** - these are named using lowercase letters but always start with a question mark. ?x is a variable and so is ?yum. This is not a convention.

**expressions** - these use the S-expression syntax a la LISP. (loves Fred Wilma) is an expression as is (friend-of Barney Fred) and (loves ?x ?y).

## Parsing

In [3]:
import tokenize
import cStringIO

This uses the above libraries to build a Lisp structure based on atoms. It is adapted from [simple iterator parser](http://effbot.org/zone/simple-iterator-parser.htm). The first function is the `atom` function.

In [4]:
def atom( next, token):
    if token[ 1] == '(':
        out = []
        token = next()
        while token[ 1] != ')':
            out.append( atom( next, token))
            token = next()
            if token[ 1] == ' ':
                token = next()
        return out
    elif token[ 1] == '?':
        token = next()
        return "?" + token[ 1]
    else:
        return token[ 1]

The next function is the actual `parse` function:

In [5]:
def parse( exp):
    src = cStringIO.StringIO( exp).readline
    tokens = tokenize.generate_tokens( src)
    return atom( tokens.next, tokens.next())

From a Python perspective, we want to turn something like "(loves Fred ?x)" to ["loves" "Fred" "?x"] and then work with the second representation as a list of strings. The strings then have the syntactic meaning we gave them previously.

In [6]:
parse("Fred")

'Fred'

In [7]:
parse( "?x")

'?x'

In [8]:
parse( "(loves Fred ?x)")

['loves', 'Fred', '?x']

In [9]:
parse( "(fatherOf Barney (sonOf Barney))")

['fatherOf', 'Barney', ['sonOf', 'Barney']]

In [10]:
def is_variable( exp):
    return isinstance( exp, str) and exp[ 0] == "?"

In [11]:
is_variable( "Fred")

False

In [12]:
is_variable( "?fred")

True

The second tests to see if an expression is a constant:

In [13]:
def is_constant( exp):
    return isinstance( exp, str) and not is_variable( exp)

In [14]:
is_constant( "Fred")

True

In [15]:
is_constant( "?fred")

False

In [16]:
is_constant( ["loves", "Fred", "?wife"])

False

It might also be useful to know that:

<code>
type( "a")
&lt;type 'str'>
type( "a") == str
True
type( "a") == list
False
type( ["a"]) == list
True
</code>

You need to write the `unification` function described above. It should work with two expressions of the type returned by `parse`. See `unify` for how it will be called. It should return the result of unification for the two expressions as detailed above and in the book.

-----

Function is_list is used to determine whether the expression is a list.

In [17]:
def is_list(expression):
    return type(expression) is list

Function check_occurrence is used to check whether the variable is the expression or in the expression.

In [18]:
def check_occurrence(variable, expression):
    if variable == expression:
        return True
    elif is_list(expression) and not is_constant(expression):
        for exp in expression:
            if check_occurrence(variable, exp): return True
    return False

Function add_substitution is used to add {var|expression} to the substitution.

In [19]:
def add_substitution(variable, expression, substitution):
    s = copy.deepcopy(substitution)
    s[variable] = expression
    return s

Function unification and unification_variable implements the unification algorithm. The algorithm works by comparing the structures of the inputs, element by element. The substitution that is the argument to the unification and unification_variable is built up along the way and is used to make sure that later comparisons are consistent with bindings that were established earlier.

In [20]:
def unification(expression1, expression2, substitution):
    if substitution == None:
        return None
    elif expression1 == expression2:
        return substitution
    elif is_variable(expression1):
        return unification_variable(expression1, expression2, substitution)
    elif is_variable(expression2):
        return unification_variable(expression2, expression1, substitution)
    elif is_constant(expression1) or is_constant(expression2) or not expression1 or not expression2:
        if (expression1 == expression2):
            return substitution
        else:
            return None
    elif is_list(expression1) and is_list(expression2):
        return unification(expression1[1:], expression2[1:], unification(expression1[0], expression2[0], substitution))
    else:
        return None

In [21]:
def unification_variable(variable, expression, substitution):
    if variable in substitution:
        return unification(substitution[variable], expression, substitution)
    elif check_occurrence(variable, expression):
        return None
    else:
        return add_substitution(variable, expression, substitution)

In [22]:
def unify(s_expression1, s_expression2):
    print parse(s_expression1)
    print parse(s_expression2)
    return unification(parse(s_expression1), parse(s_expression2), {})

Using `unify`, solve the problems in the self-check. **Important** use `unify` **not** `unification`. Do one expression per cell:

In [23]:
print unify("(Fred)", "(Barney)")

['Fred']
['Barney']
None


In [24]:
print unify("(Pebbles)", "(Pebbles)")

['Pebbles']
['Pebbles']
{}


In [25]:
print unify("(quarry Fred)", "(quarry ?x)")

['quarry', 'Fred']
['quarry', '?x']
{'?x': 'Fred'}


In [26]:
print unify("(son Barney ?x)", "(son ?y BamBam)")

['son', 'Barney', '?x']
['son', '?y', 'BamBam']
{'?y': 'Barney', '?x': 'BamBam'}


In [27]:
print unify("(married ?x ?y)", "(married Barney Wilma)")

['married', '?x', '?y']
['married', 'Barney', 'Wilma']
{'?y': 'Wilma', '?x': 'Barney'}


In [28]:
print unify("(son Barney ?x)", "(son ?y (son Barney))")

['son', 'Barney', '?x']
['son', '?y', ['son', 'Barney']]
{'?y': 'Barney', '?x': ['son', 'Barney']}


In [29]:
print unify("(son Barney Bambam)", "(son ?y (son Barney))")

['son', 'Barney', 'Bambam']
['son', '?y', ['son', 'Barney']]
None


In [30]:
print unify("(loves Fred Fred)", "(loves ?x ?x)")

['loves', 'Fred', 'Fred']
['loves', '?x', '?x']
{'?x': 'Fred'}


In [31]:
print unify("(future George Fred)", "(future ?y ?y)")

['future', 'George', 'Fred']
['future', '?y', '?y']
None


In [33]:
print unify("(son Barney ?x)", "(son ?y (son ?y))")

['son', 'Barney', '?x']
['son', '?y', ['son', '?y']]
{'?y': 'Barney', '?x': ['son', '?y']}
